# ***CS 3501: PS2***

## **1. Maximum Likelihood Estimation (MLE)**

$$p(x;\lambda) = 2 \lambda x \text{exp}(- \lambda x^2)$$


a)

Step 1: Likelihood function for $p(x;\lambda)$
$$L( \lambda ) = \Pi(2\lambda\text{exp}(-\lambda x^2))$$

Step 2: Handle $\Pi$
$$L( \lambda) = (2\lambda)^n \text{exp}(-\lambda \Sigma{x_i^2})$$

Step 3: Take the natural log of both sides and simplify
$$l(\lambda) = \text{ln}((2\lambda)^n \text{exp}(-\lambda \Sigma{x_i^2}))$$
$$l(\lambda) = \text{ln}((2\lambda)^n) + -\lambda \Sigma{x_i^2}$$
$$l(\lambda) = n\text{ln}(2) + n\text{ln}(\lambda) - \lambda \Sigma{x_i^2}$$

Step 4: Calculate the derivative
$$\frac{dl}{d \lambda} = 0 + \frac{n}{\lambda} - \Sigma{x_i^2}$$
$$\frac{dl}{d \lambda} = \frac{n}{\lambda} - \Sigma{x_i^2}$$

Step 5: Solve when the derivative is equal to $0$
$$0 = \frac{n}{\lambda} - \Sigma{x_i^2}$$
$$\Sigma{x_i^2} = \frac{n}{\lambda}$$
$$\lambda = \frac{n}{\Sigma{x_i^2}}$$

Therefore, $\lambda = \frac{n}{\Sigma{x_i^2}}$.

b)


In [1]:
import pandas as pd

traffic_dataset = pd.read_csv("traffic.csv")

c)

## **2. Hypothesis Testing**

## **3. K-means**

## **Bonus Points**